In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import keras.backend as K
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization ,Activation
from tensorflow.keras.models import Model, Sequential
from keras.applications.nasnet import NASNetLarge
import math
import cv2
os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Models

In [ ]:
model1 = tf.keras.applications.ResNet50(input_shape=(224,224,3),include_top=True,weights=None,classes=7)
model1.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
model1.load_weights('../input/emotion-util/ResNet_continue.h5')

In [ ]:
model2 = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),include_top=True,weights=None,classes=7)
model2.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
model2.load_weights('../input/emotion-util/MobileNet.h5')

In [ ]:
model3 = tf.keras.applications.InceptionV3(input_shape=(224,224,3),include_top=True,weights=None,classes=7)
model3.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
model3.load_weights('../input/emotion-util/InceptionNet.h5')

In [ ]:
model4 = tf.keras.models.load_model('../input/landmark-model/lm_model')

In [ ]:
df = pd.read_csv('../input/224-landmarks/feature_extra.csv')
test = df[df['Usage']=='PrivateTest'].copy().reset_index()
X_img = np.zeros([test.shape[0],224,224])

### Prepare Data 
(We have saved the result in 224_lanmarks, this step can be skipped)

In [ ]:
def bilinear_resize(image, height, width):
    img_height, img_width = image.shape[:2]
    resized = np.empty([height, width])
    x_ratio = float(img_width - 1) / (width - 1) if width > 1 else 0
    y_ratio = float(img_height - 1) / (height - 1) if height > 1 else 0

    for i in range(height):
        for j in range(width):
            x_l, y_l = math.floor(x_ratio * j), math.floor(y_ratio * i)
            x_h, y_h = math.ceil(x_ratio * j), math.ceil(y_ratio * i)

            x_weight = (x_ratio * j) - x_l
            y_weight = (y_ratio * i) - y_l

            a = image[y_l, x_l]
            b = image[y_l, x_h]
            c = image[y_h, x_l]
            d = image[y_h, x_h]

            pixel = a * (1 - x_weight) * (1 - y_weight) + b * x_weight * (1 - y_weight) +  c * y_weight * (1 - x_weight) + d * x_weight * y_weight

            resized[i][j] = pixel

    return resized

In [ ]:
for i in range(test.shape[0]):
    img= np.array([int(x) for x in test.loc[i]['pixels'].split()]).reshape(48,48)
    img = bilinear_resize(img, 224, 224)
    X_img[i] = img

In [ ]:
def generate_X(df):
    X = np.empty([df.shape[0],176])
    for i in range(df.shape[0]):
        lm = [float(x) for x in df.loc[i]['relative_lm'].split()][:136]

        brow_dis = [float(x) for x in df.loc[i]['brow_distance'].split()]
        
        eye_dis = [float(x) for x in df.loc[i]['eyes_distance'].split()]

        mouth_dis = [float(x) for x in df.loc[i]['mouth_distance'].split()]

        mouth_width = [float(df.loc[i]['mouth_width'])]
        mouth_height = [float(df.loc[i]['mouth_height'])]
        mouth_upward = [float(x) for x in df.loc[i]['mouth_upward'].split()]
        
        all_feature = lm+brow_dis+eye_dis+mouth_dis+mouth_width+mouth_height+mouth_upward
        
        X[i,:] = all_feature
    return X

X_lm = generate_X(test)

In [ ]:
np.save('X_img.npy',X_img)
np.save('X_lm.npy',X_lm)

### Fuse 4 models

In [ ]:
# load the saved test data
X_img = np.load("../input/224-landmarks/X_img.npy").astype('uint8') # for image based model
X_lm = np.load("../input/224-landmarks/X_lm.npy") # for landmark based model
X_img_3 = np.array([cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) for img in X_img]) #change the image from Gray to RGB

In [ ]:
X_img_3 = X_img_3/255

from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_lm)
X_lm = scaler.transform(X_lm)

In [ ]:
# prediction result
y_pred_1 = model1.predict(X_img_3)
y_pred_2 = model2.predict(X_img_3)
y_pred_3 = model3.predict(X_img_3)
y_pred_4 = model4.predict(X_lm)

### Evaluation

In [ ]:
# because landmark_model and image based model use different label method, here we change the label. 
y_pred_4 = np.where(y_pred_4 == 4,99, y_pred_4)
y_pred_4 = np.where(y_pred_4 == 5,100, y_pred_4)
y_pred_4 = np.where(y_pred_4 == 6,4, y_pred_4)
y_pred_4 = np.where(y_pred_4 == 99,5, y_pred_4)
y_pred_4 = np.where(y_pred_4 == 100,6, y_pred_4)

In [ ]:
Y_pred = np.argmax(y_pred_1+y_pred_2+y_pred_3+y_pred_4,axis=1)

# because landmark_model and image based model use different label method, here we change the label.
Y_test = np.array(test['emotion'])
Y_test = np.where(Y_test == 4,99, Y_test)
Y_test = np.where(Y_test == 5,100, Y_test)
Y_test = np.where(Y_test == 6,4, Y_test)
Y_test = np.where(Y_test == 99,5, Y_test)
Y_test = np.where(Y_test == 100,6, Y_test)

In [ ]:
# prediction result for 3 classes:
# positve: happy, surprise, negative: anger, fear, disgust, sad, neutral
y_test_3c =  np.where(Y_test== 1,0,Y_test)
y_test_3c =  np.where(y_test_3c== 2, 0,y_test_3c)
y_test_3c =  np.where(y_test_3c== 3, 1,y_test_3c)
y_test_3c =  np.where(y_test_3c== 4, 2,y_test_3c)
y_test_3c =  np.where(y_test_3c== 5, 0,y_test_3c)
y_test_3c =  np.where(y_test_3c== 6, 1,y_test_3c)

y_pred_3c =  np.where(Y_pred== 1,0,Y_pred)
y_pred_3c =  np.where(y_pred_3c== 2, 0,y_pred_3c)
y_pred_3c =  np.where(y_pred_3c== 3, 1,y_pred_3c)
y_pred_3c =  np.where(y_pred_3c== 4, 2,y_pred_3c)
y_pred_3c =  np.where(y_pred_3c== 5, 0,y_pred_3c)
y_pred_3c =  np.where(y_pred_3c== 6, 1,y_pred_3c)

In [ ]:
# calculate prediction accuracy
(y_pred_3c  == y_test_3c).sum()/2579

In [ ]:
import seaborn as sns
from sklearn import metrics

cm1 = metrics.confusion_matrix(Y_pred, Y_test)
cm_normalized = cm1.astype('float') / cm1.sum(axis=1)[:, np.newaxis]
labels = ['anger','disgust','fear','happy','neutral','sad','surprise']
# labels = ['negative','positive','nutreul']

plt.figure(figsize=(9,9))
sns.heatmap(cm_normalized, annot=True, fmt=".2f", linewidths=.5, square=True, cmap='Blues',
            cbar_kws={"shrink": .82},annot_kws={"size":20})
plt.ylabel('True emotion', size=17,rotation = 90)
tick_marks = np.array(range(len(labels))) + 0.5
plt.xticks(tick_marks,labels,ha='center',fontsize = 12)
plt.yticks(tick_marks,labels,va='center',fontsize = 12)
plt.xlabel('Predicted emotion', size=14)
plt.show()